In [6]:
from modelzipper.tutils import *
import pickle

# 读取 .pkl 文件
with open("/mnt/petrelfs/tangzecheng/MyRLHF/evaluation/babilong/llama-3_1-8B-Instruct/preliminary/preds_llama-3_1-8B-Instruct_top_n.jsonl.pkl", "rb") as file:  # 使用 "rb" 模式读取二进制文件
    loaded_data = pickle.load(file)

ConnectionRefusedError: [Errno 111] Connection refused